In [1]:
import pandas as pd
import os

En este código vamos a crear una base con los actores, directores y puntuaciones para luego juntar a la base de IMDB que usaremos para el proyecto.

In [2]:


# Directorio donde se encuentran los archivos TSV
directorio = "data/Base oficial IMDB"

# Cargando los archivos TSV en DataFrames
name_basics = pd.read_csv(os.path.join(directorio, "name.basics.tsv"), sep='\t')
title_akas = pd.read_csv(os.path.join(directorio, "title.akas.tsv"), sep='\t')
title_basics = pd.read_csv(os.path.join(directorio, "title.basics.tsv"), sep='\t')
title_crew = pd.read_csv(os.path.join(directorio, "title.crew.tsv"), sep='\t')
title_episode = pd.read_csv(os.path.join(directorio, "title.episode.tsv"), sep='\t')
title_principals = pd.read_csv(os.path.join(directorio, "title.principals.tsv"), sep='\t')
title_ratings = pd.read_csv(os.path.join(directorio, "title.ratings.tsv"), sep='\t')



C:\Users\mpata_lrc8dz0\AppData\Local\Temp\ipykernel_17816\2066758021.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics = pd.read_csv(os.path.join(directorio, "title.basics.tsv"), sep='\t')


In [3]:

#Vamos a crear un solo dataframe con la información de las películas que necesito
movies = title_basics[(title_basics['titleType'] == 'movie') & (title_basics['isAdult'].isin([0, '0']))]

# Obtener títulos en español para busquedas
titles_es = title_akas[(title_akas['region'] == 'ES')][['titleId', 'title']]

#Ahora voy a sumar el título en español al DataFrame movies si lo tenemos.
movies = pd.merge(movies, titles_es[['titleId', 'title']].rename(columns={'title': 'title_es'}),
                  left_on='tconst', right_on='titleId', how='left')

movies.drop('titleId', axis=1, inplace=True)

# Visualizar las primeras filas del DataFrame movies para verificar
print(movies.head())


      tconst titleType                   primaryTitle  \
0  tt0000009     movie                     Miss Jerry   
1  tt0000147     movie  The Corbett-Fitzsimmons Fight   
2  tt0000502     movie                       Bohemios   
3  tt0000574     movie    The Story of the Kelly Gang   
4  tt0000591     movie               The Prodigal Son   

                   originalTitle isAdult startYear endYear runtimeMinutes  \
0                     Miss Jerry       0      1894      \N             45   
1  The Corbett-Fitzsimmons Fight       0      1897      \N            100   
2                       Bohemios       0      1905      \N            100   
3    The Story of the Kelly Gang       0      1906      \N             70   
4              L'enfant prodigue       0      1907      \N             90   

                       genres  title_es  
0                     Romance       NaN  
1      Documentary,News,Sport       NaN  
2                          \N  Bohemios  
3  Action,Adventure,Biogra

In [4]:
# Unir información de ratings
movies = movies.merge(title_ratings[['tconst', 'averageRating', 'numVotes']], on='tconst', how='left')


In [5]:
# Obtener los actores principales
principals_actors = title_principals[(title_principals['category'].isin(['actor', 'actress'])) & (title_principals['tconst'].isin(movies['tconst']))]
# Supongamos que solo queremos los nombres y los uniremos en una lista separada por comas
actors_names = principals_actors.merge(name_basics[['nconst', 'primaryName']], left_on='nconst', right_on='nconst')
actors_summary = actors_names.groupby('tconst')['primaryName'].apply(lambda x: ', '.join(x)).reset_index()
movies = movies.merge(actors_summary, on='tconst', how='left')

In [6]:
#Completo el director de la película
movies = movies.merge(title_crew[['tconst', 'directors']], on='tconst', how='left')

movies_directors = movies[['tconst', 'directors']]
# Voy a convertir la información de los directores en una lista y quitar los nulos
movies_directors['directors'] = movies_directors['directors'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
# Me voy a crear una tabla con id y directores para poder completar todo el proceso
movies_exploded = movies_directors.explode('directors')

movies_directors.head()

C:\Users\mpata_lrc8dz0\AppData\Local\Temp\ipykernel_17816\3948828377.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_directors['directors'] = movies_directors['directors'].apply(lambda x: x.split(',') if isinstance(x, str) else [])


,tconst,directors
0,tt0000009,[nm0085156]
1,tt0000147,[nm0714557]
2,tt0000502,[nm0063413]
3,tt0000574,[nm0846879]
4,tt0000591,[nm0141150]


In [7]:
#Ahora que ya tengo todo preparado voy a unir la información de los directores en movies.
movies_exploded = movies_directors.explode('directors')

# Junto movies_exploded con name_basics para obtener los nombres de los directores no solo sus ID
movies_with_names = pd.merge(
    movies_exploded,
    name_basics[['nconst', 'primaryName']],
    left_on='directors',
    right_on='nconst',
    how='left'
)


#Ahora agrupo otra vez para juntar el nombre de los directores en cada película
#Los voy a guardar si hay más de uno como los actores separando por comas.
movies_with_names_grouped = movies_with_names.groupby('tconst').agg({
    'primaryName': lambda x: ', '.join(x.dropna().unique())  
}).reset_index()

# Cambio el nombre a primaryName a directors_name
movies_with_names_grouped.rename(columns={'primaryName': 'directors_name'}, inplace=True)

# Creo finalmente movies_final con los nombres de los directores
movies_final = pd.merge(
    movies_directors,
    movies_with_names_grouped[['tconst', 'directors_name']],
    on='tconst',
    how='left'
)





In [8]:
#Finalmente añado la columna de nombre de director al DataFrame movies
movies = movies.merge(movies_final[['tconst', 'directors_name']], on='tconst', how='left')


In [19]:
#Esto lo creo para poder buscar una peli y ver si los directores son los correctos
titulo = input("Introduce el título de la película: ")

# Filtro movies usando primaryTitle y title_es
peliculas_filtradas = movies[(movies['primaryTitle'].str.contains(titulo, case=False, na=False)) |
                             (movies['title_es'].str.contains(titulo, case=False, na=False))]

print("\nPelículas encontradas:")
print(peliculas_filtradas[['tconst', 'primaryTitle', 'title_es', 'directors_name','primaryName']].head(10))


Películas encontradas:
         tconst                          primaryTitle  \
681   tt0004406                       Noche de sangre   
2208  tt0007297                        Sangre y arena   
3436  tt0009039  La España trágica o Tierra de sangre   
3864  tt0009580                      The Scarlet Drop   
6565  tt0012890                           Alma rifeña   
6566  tt0012890                           Alma rifeña   
6620  tt0012952                        Blood and Sand   
7445  tt0013921                            Chî to reî   
9248  tt0015973                        Kentucky Pride   
9529  tt0016311             Sangre azul y sangre roja   

                        title_es                           directors_name  \
681              Noche de sangre          Ricardo de Baños, Alberto Marro   
2208              Sangre y arena  Ricardo de Baños, Vicente Blasco Ibáñez   
3436  La narracion de un soldado                          Rafael Salvador   
3864           La gota de sangre        

In [10]:
#Aquí voy a eliminar las columnas que no me valen.
movies.drop(['titleType', 'isAdult', 'directors', 'endYear'], axis=1, inplace=True)


In [39]:
movies.info()   

<class 'pandas.core.frame.DataFrame'>
Index: 670198 entries, 0 to 700739
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          670198 non-null  object 
 1   primaryTitle    670198 non-null  object 
 2   originalTitle   670198 non-null  object 
 3   startYear       670198 non-null  Int64  
 4   runtimeMinutes  670198 non-null  Int64  
 5   genres          670198 non-null  object 
 6   title_es        67648 non-null   object 
 7   averageRating   304877 non-null  float64
 8   numVotes        304877 non-null  float64
 9   primaryName     485787 non-null  object 
 10  directors_name  670198 non-null  object 
dtypes: Int64(2), float64(2), object(7)
memory usage: 62.6+ MB


In [12]:
#Últimos ajustes.
# vamos a borrar las filas donde primaryTitle sea nulo
movies = movies.dropna(subset=['primaryTitle'])


In [32]:
movies = movies.drop_duplicates(subset=['tconst'])

In [38]:
#Vamos a corregir el formato de algunas columnas, comenzamos por startYear, quiero que sea entero 
movies['startYear'] = movies['startYear'].replace('\\N', '0')
movies['startYear'] = movies['startYear'].astype('Int64')
#Luego haremos los mismo con runtimeMinutes
movies['runtimeMinutes'] = movies['runtimeMinutes'].replace('\\N', '0')
movies['runtimeMinutes'] = movies['runtimeMinutes'].astype('Int64')



In [40]:
#Voya cambiar el nombre de algunas columnas por ejemplo primaryTitle por title, startYear por year, primaryName por actors
movies.rename(columns={'primaryTitle': 'title', 'startYear': 'year', 'primaryName': 'actors'}, inplace=True)


In [41]:
#por último me exporto a csv el dataframe movies
movies.to_csv('movies.csv', index=False)
